<a href="https://colab.research.google.com/github/mralamdari/CV-Yolo/blob/main/Yolo_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import PIL
import struct # used to convert native Python data types such as strings and numbers into a string of bytes and vice versa
import colorsys
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tensorflow.python.saved_model import tag_constants

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d aruchomu/data-for-yolo-v3-kernel
!unzip \*.zip && rm *.zip

In [ ]:
!wget -P model_data https://pjreddie.com/media/files/yolov3.weights

!wget -P model_data https://pjreddie.com/media/files/yolov3-tiny.weights

!wget -P model_data https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

!wget -P model_data https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

# Load The Yolo Model

choose between yolo frameworks between; tf and trt


and yolo types between;
yolov3
yolov3-tiny
yolov4
yolov4-tiny


 and decide if you need custom weights 

In [ ]:
def load_yolo_model(yolo_framework, yolo_type, yolo_costom_weights, input_size, input_classes, class_names):
    
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        pass

    if yolo_framework == "tf": # TensorFlow detection framework
        
        if yolo_costom_weights:
            checkpoint = f"./checkpoints/{yolo_type}_custom"
            print(f"Loading custom weights from: {checkpoint}")
            yolo = create_yolo_model(yolo_type, class_names, input_size=input_size, classes=input_classes)
            yolo.load_weights(checkpoint)
        else:
            Darknet_weights = f'model_data/{yolo_type}.weights'
            print(f"Loading Darknet_weights from: {Darknet_weights}")
            yolo = create_yolo_model(yolo_type, class_names, input_size=input_size, classes=input_classes)
            load_yolo_weights(yolo, Darknet_weights) # use Darknet weights
        
    elif yolo_framework == "trt": # TensorRT detection framework
        saved_model_loaded = tf.saved_model.load(yolo_costom_weights, tags=[tag_constants.SERVING])
        signature_keys = list(saved_model_loaded.signatures.keys())
        yolo = saved_model_loaded.signatures['serving_default']

    return yolo 

# Create Yolo Model


In [ ]:
def create_yolo_model(yolo_type, class_names, classes, input_size=416, channels=3, training=False):
    
    num_classes = len(class_names)
    input_layer  = tf.keras.layers.Input([input_size, input_size, channels])

    if yolo_type[-4:] == 'tiny':
        if yolo_type == "yolov4":
            convolutional_layers = YOLOv4_tiny(input_layer, num_classes)
        if yolo_type == "yolov3":
            convolutional_layers = YOLOv3_tiny(input_layer, num_classes)
    else:
        if yolo_type == "yolov4":
            convolutional_layers = YOLOv4(input_layer, num_classes)
        if yolo_type == "yolov3":
            convolutional_layers = YOLOv3(input_layer, num_classes)

    output_tensors = []
    for i, conv_layer in enumerate(convolutional_layers):
        pred_tensor = decode(conv_layer, num_classes, i)
        
        if training: 
          output_tensors.append(conv_layer)
        
        output_tensors.append(pred_tensor)

    Yolo = tf.keras.Model(input_layer, output_tensors)

    return Yolo

# Up Sample

resize the batch of images' height and weidth

    # shape=(None, 13, 13, 256)    ===>   (None, 26, 26, 256)
    # shape=(None, 26, 26, 128)    ===>   (None, 56, 56, 128)

In [ ]:
def upsample(input_layer):
    return tf.keras.layers.UpSampling2D(2)(input_layer)
    # return tf.image.resize(input_layer, (input_layer.shape[1] * 2, input_layer.shape[2] * 2), method='nearest')

# Convolutional Layer

In [ ]:
def convolutional(input_layer, filters_shape, downsample=False, activate=True, bn=True, activate_type='leaky'):
    if downsample:
        input_layer = tf.keras.layers.ZeroPadding2D(((1, 0), (1, 0)))(input_layer)
        padding = 'valid'
        strides = 2
    else:
        strides = 1
        padding = 'same'

    conv = tf.keras.layers.Conv2d(filters=filters_shape[-1],
                                  kernel_size=filters_shape[0],
                                  strides=strides,
                                  padding=padding,
                                  use_bias=not bn,
                                  kernel_regularizer=tf.keras.regularizers.L2(0.0005),
                                  kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                  bias_initializer=tf.constant_initializer(0.))(input_layer)  

    if bn: # BatchNormalization
        conv = tf.keras.layers.BatchNormalization()(conv)
    if activate == True: # Activation
        if activate_type == "leaky":
            conv = tf.keras.layers.LeakyReLU(alpha=0.1)(conv)
        elif activate_type == "mish":
          conv = tf.math.softplus(conv)
          conv = conv * tf.math.tanh(conv)

    return conv 

# Residual Block

In [ ]:
def residual_block(x, channels, filter1, filter2, activation='leaky'):
    shortcut = x
    x = convolutional(x, filters_shape=(1, 1, channels,filter1), activate_type=activation)
    x = convolutional(x, filters_shape=(3, 3, filter1, filter2), activate_type=activation)

    residual_layer = shortcut + x
    return residual_layer

In [ ]:
# yolo_framework    ====> 'tf', 'trt'   
# yolo_type         ====> yolov3, yolov4, yolov3-tiny, yolov4-tiny
# input_classes     ====> "mnist/mnist.names", "/content/coco.names"

input_classes="/content/coco.names"

class_names = {}
with open(input_classes, 'r') as data:
    for ID, name in enumerate(data):
        class_names[ID] = name.strip('\n')

yolo = Load_Yolo_model(yolo_framework='tf',
                       yolo_type='yolov3',
                       yolo_costom_weights=False,
                       input_size=416,
                       input_classes=input_classes, 
                       class_names=class_names)

image_path = '/content/dog.jpg'
image_path = '/content/office.jpg'